In [1]:
import pandas as pd
import numpy as np
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from collections import Counter
import re
import random
from tqdm import tqdm
import ast

2025-04-28 10:54:48.167618: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745837688.350310      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745837688.402927      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# Load original preprocessed data
data = pd.read_csv('/kaggle/input/kubernetes-bug-to-augment-2/train_data.csv')

In [3]:
data.head()

,comments_url,id,title,body,issue_url,pr_url,labels,pr_number,filename,status,additions,deletions,changes,all_comments,all_text
0,https://api.github.com/repos/kubernetes/kubern...,2639668210,kubelet crash: fatal error: concurrent map writes,### What happened?\n\nWhile looking into three...,https://github.com/kubernetes/kubernetes/issue...,https://github.com/kubernetes/kubernetes/pull/...,['area/kubelet'],128657,"['pkg/kubelet/cm/container_manager_linux.go', ...","['modified', 'modified', 'modified', 'modified']","[3, 4, 18, 75]","[3, 3, 6, 0]","[6, 7, 24, 75]",/sig node Thought to search for similar errors...,kubelet fatal concurrent map writes while look...
1,https://api.github.com/repos/kubernetes/kubern...,2617512099,[FG:InPlacePodVerticalScaling] failed to verif...,### What happened?\n\nOne line bug description...,https://github.com/kubernetes/kubernetes/issue...,https://github.com/kubernetes/kubernetes/pull/...,['area/kubelet'],126620,"['pkg/kubelet/status/state/checkpoint.go', 'pk...","['modified', 'modified', 'added']","[42, 23, 166]","[26, 17, 0]","[68, 40, 166]",This issue is currently awaiting triage.\n\nIf...,failed verify pod status checkpoint checksum b...
2,https://api.github.com/repos/kubernetes/kubern...,2604613192,Restore build-tag flag for code-generator,### What happened?\n\nThe `build-tag` flag is ...,https://github.com/kubernetes/kubernetes/issue...,https://github.com/kubernetes/kubernetes/pull/...,['area/code-generation'],128259,['staging/src/k8s.io/code-generator/cmd/conver...,"['modified', 'modified', 'modified', 'modified...","[12, 1, 1, 12, 1, 1]","[2, 1, 1, 1, 1, 1]","[14, 2, 2, 13, 2, 2]",@p0lyn0mial FYI /sig api-machinery /triage acc...,restore flag flag removed moved removed depend...
3,https://api.github.com/repos/kubernetes/kubern...,2596132738,[Failing Tests] ci-crio-cgroupv1-node-e2e-conf...,### Which jobs are failing?\n* master-blocking...,https://github.com/kubernetes/kubernetes/issue...,https://github.com/kubernetes/kubernetes/pull/...,"['area/kubeadm', 'area/dependency']",128175,"['go.mod', 'go.sum', 'hack/unwanted-dependenci...","['modified', 'modified', 'modified', 'modified...","[1, 3, 1, 21, 9, 9, 4, 0, 10, 2, 2, 5, 8, 1]","[1, 2, 0, 98, 8, 7, 6, 62, 11, 1, 4, 8, 11, 1]","[2, 5, 1, 119, 17, 16, 10, 62, 21, 3, 6, 13, 1...",@drewhagen: The provided milestone is not vali...,multiple which jobs are which tests are node c...
4,https://api.github.com/repos/kubernetes/kubern...,2591358936,Crash on kube manager's service-lb-controller ...,### What happened?\n\nIf kube manager is start...,https://github.com/kubernetes/kubernetes/issue...,https://github.com/kubernetes/kubernetes/pull/...,['area/cloudprovider'],128182,['cmd/kube-controller-manager/app/controllerma...,"['modified', 'modified', 'modified']","[26, 12, 5]","[0, 1, 4]","[26, 13, 9]",This issue is currently awaiting triage.\n\nIf...,crash kube after kube manager started no looks...


In [4]:
# Flatten the list of labels if it's a list of lists
flattened_labels = []
for label_list in data["labels"]:
    # Convert string representation of list to actual list
    if isinstance(label_list, str):
        # Remove brackets and split by comma
        clean_label_list = label_list.strip('[]').replace("'", "").split(', ')
    else:
        clean_label_list = label_list
    
    # Add each label to the flattened list
    flattened_labels.extend(clean_label_list)

# Print labels distribution count
label_counts = Counter(flattened_labels)
print("Label Distribution Count:")
for label, count in label_counts.most_common():
    print(f"{label}: {count}")


Label Distribution Count:
area/test: 309
area/kubelet: 304
area/apiserver: 196
area/cloudprovider: 171
area/kubectl: 133
area/dependency: 80
area/code-generation: 65
area/provider/azure: 64
area/ipvs: 39
area/kubeadm: 33
area/provider/gcp: 32
area/e2e-test-framework: 28
area/kube-proxy: 27
area/release-eng: 23
area/conformance: 12
area/custom-resources: 6
area/security: 5
area/etcd: 5
area/deflake: 3
area/network-policy: 3
area/code-organization: 2
area/api: 1
area/batch: 1
area/client-libraries: 1
area/provider/aws: 1


In [5]:
# Define target count for each label (aiming for better balance)
# We'll use the median count as a reference point
median_count = np.median(list(label_counts.values()))
# Set a target threshold of at least 100 samples per label or 75% of the top label count
target_threshold = max(100, int(max(label_counts.values()) * 0.75))
# target_threshold = 10
print(f"\nTarget threshold for augmentation: {target_threshold}")


Target threshold for augmentation: 231


In [6]:
# Function to load GPT-2 model
def load_gpt2_model():
    print("Loading GPT-2 model...")
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    # Set pad token to the EOS token
    tokenizer.pad_token = tokenizer.eos_token
    model = GPT2LMHeadModel.from_pretrained('gpt2')
    model.config.pad_token_id = model.config.eos_token_id
    if torch.cuda.is_available():
        if torch.cuda.device_count() > 1:
            print(f"Using {torch.cuda.device_count()} GPUs!")
            model = torch.nn.DataParallel(model) # Wrap the model for multi-GPU
        model = model.cuda() # Move the wrapped model to GPU(s)
    return model, tokenizer

# Function to generate text using GPT-2
def generate_text_with_gpt2(model, tokenizer, prompt, max_length=512):
    inputs = tokenizer.encode_plus(
        prompt,
        return_tensors='pt',
        add_special_tokens=True,
        return_attention_mask=True,
        padding='max_length',
        max_length=min(len(prompt.split()) + 10, 1024)  # Reasonable context length
    )
    
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']
    
    # Move tensors to the same device as the model (GPU 0 if using DataParallel)
    if torch.cuda.is_available():
        device = next(model.parameters()).device # Get device from model
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
    
    # Generate text
    # Access the underlying model's generate method if using DataParallel
    if isinstance(model, torch.nn.DataParallel):
        generate_func = model.module.generate
    else:
        generate_func = model.generate
        
    outputs = generate_func(
        input_ids,
        attention_mask=attention_mask,
        max_length=max_length, # Use the function parameter
        temperature=0.9,
        top_k=50,
        top_p=0.95,
        repetition_penalty=1.2,
        do_sample=True,
        num_return_sequences=1,
        pad_token_id=tokenizer.pad_token_id
    )
    
    # Decode the generated text
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text

# Function to clean the generated text
def clean_generated_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9 ]', '', text)
    return text

# Generate prompts for each label from existing data
def create_prompts_by_label(data):
    prompts_by_label = {}
    
    for label in tqdm(label_counts.keys(), desc="Creating prompts by label"):
        # Get samples containing this label
        label_samples = data[data['labels'].apply(lambda x: label in x)]
        
        if not label_samples.empty:
            # Select some samples (up to 5) to use as seeds for generation
            selected_samples = label_samples.sample(min(5, len(label_samples)))
            prompts_by_label[label] = selected_samples['all_text'].tolist()
    
    return prompts_by_label

# Augment the dataset
def augment_with_gpt2(data, label_counts, target_threshold):
    model, tokenizer = load_gpt2_model()
    model.eval()  # Set to evaluation mode
    
    # Create prompts for each label
    prompts_by_label = create_prompts_by_label(data)
    
    # Create new dataframe to store augmented data
    augmented_samples = []
    
    print("\nAugmenting underrepresented labels:")
    for label, count in sorted(label_counts.items(), key=lambda x: x[1]):

        # Only augment labels below the threshold
        if count < target_threshold:
            # Calculate how many samples to generate
            num_to_generate = target_threshold - count
            print(f"  Starting generation of {num_to_generate} samples for {label} (current count: {count})")
            
            # Generate samples with progress bar
            for i in tqdm(range(num_to_generate), desc=f"  Generating samples for {label}", leave=False):
                # Choose a random prompt for this label
                if label in prompts_by_label and len(prompts_by_label[label]) > 0:
                    prompt = random.choice(prompts_by_label[label])
                    
                    # Generate text
                    generated_text = generate_text_with_gpt2(model, tokenizer, prompt[:100])
                    cleaned_text = clean_generated_text(generated_text)
                    
                    # Create a sample
                    # Split the text into title and body (first 10 words for title, rest for body)
                    words = cleaned_text.split()
                    title = ' '.join(words[:10])
                    body = ' '.join(words[10:])
                    
                    # Add to augmented samples
                    augmented_samples.append({
                        'id': f"augmented_{len(augmented_samples)}",
                        'title': title,
                        'body': body,
                        'labels': [label],  # Only include the label we're augmenting for
                        'all_text': cleaned_text,
                        'is_augmented': True  # Flag to identify synthetic samples
                    })
    
    # Create dataframe with augmented samples
    augmented_df = pd.DataFrame(augmented_samples)
    
    # Combine with original data
    # First, add is_augmented column to original data
    data['is_augmented'] = False
    
    # Concatenate dataframes
    if len(augmented_samples) > 0:
        combined_data = pd.concat([data, augmented_df], ignore_index=True)
    else:
        combined_data = data.copy()
    
    return combined_data

In [7]:
# Augment the dataset
augmented_data = augment_with_gpt2(data, label_counts, target_threshold)

# Process the labels 
new_flattened_labels = []
for label_list in augmented_data['labels']:
    # Convert string representation of list to actual list
    if isinstance(label_list, str):
        try:
            # Safely evaluate the string as a Python literal (list)
            evaluated_list = ast.literal_eval(label_list)
            if isinstance(evaluated_list, list):
                clean_label_list = [str(item).strip() for item in evaluated_list]
            else:
                clean_label_list = [label_list.strip()]
        except (ValueError, SyntaxError):
            clean_label_list = [label_list.strip()]
    elif isinstance(label_list, (list, tuple)):
        clean_label_list = [str(item).strip() for item in label_list]
    else:
        clean_label_list = [str(label_list).strip()]
    
    # Add each valid label to the flattened list (filter out empty strings)
    new_flattened_labels.extend([label for label in clean_label_list if label])

new_label_counts = Counter(new_flattened_labels)

print("\nNew Label Distribution After Augmentation:")
for i, (label, count) in enumerate(sorted(new_label_counts.items(), key=lambda x: x[1], reverse=True)):
    original_count = label_counts.get(label, 0)
    print(f"{i+1}. {label}: {count} (added {count - original_count})")

# Save augmented dataset
output_path = 'augmented_data_with_gpt2.csv'
augmented_data.to_csv(output_path, index=False)
print(f"\nAugmented dataset saved to {output_path}")

# Print statistics
original_samples = len(data)
augmented_samples = len(augmented_data) - original_samples
print(f"\nStatistics:")
print(f"Original samples: {original_samples}")
print(f"Added samples: {augmented_samples}")
print(f"Total samples: {len(augmented_data)}")
print(f"Augmentation ratio: {augmented_samples/original_samples:.2%}")

Loading GPT-2 model...


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

  Starting generation of 151 samples for area/dependency (current count: 80)


  Starting generation of 98 samples for area/kubectl (current count: 133)


  Starting generation of 60 samples for area/cloudprovider (current count: 171)


  Starting generation of 35 samples for area/apiserver (current count: 196)



New Label Distribution After Augmentation:
1. area/test: 309 (added 0)
2. area/kubelet: 304 (added 0)
3. area/code-generation: 231 (added 166)
4. area/kubeadm: 231 (added 198)
5. area/dependency: 231 (added 151)
6. area/cloudprovider: 231 (added 60)
7. area/apiserver: 231 (added 35)
8. area/api: 231 (added 230)
9. area/e2e-test-framework: 231 (added 203)
10. area/kubectl: 231 (added 98)
11. area/kube-proxy: 231 (added 204)
12. area/release-eng: 231 (added 208)
13. area/ipvs: 231 (added 192)
14. area/provider/gcp: 231 (added 199)
15. area/conformance: 231 (added 219)
16. area/batch: 231 (added 230)
17. area/deflake: 231 (added 228)
18. area/provider/azure: 231 (added 167)
19. area/network-policy: 231 (added 228)
20. area/client-libraries: 231 (added 230)
21. area/code-organization: 231 (added 229)
22. area/security: 231 (added 226)
23. area/etcd: 231 (added 226)
24. area/custom-resources: 231 (added 225)
25. area/provider/aws: 231 (added 230)

Augmented dataset saved to augmented_data_